In [4]:
from lifelong_forests import *
import matplotlib.pyplot as plt

n_tasks = 10 # should divide 100 evenly
# K = int(len(class_idx)/n_tasks)

import pickle
def unpickle(file):
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

def homogenize_labels(a):
    u = np.unique(a)
    return np.array([np.where(u == i)[0][0] for i in a])

In [99]:
class LifelongForest:
    """
    Lifelong Forest class.
    """
    def __init__(self, acorn=None):
        """
        Two major things the Forest Class needs access to:
            1) the realized random forest model (self.models_ is a list of forests, 1 for each task)
            2) old data (to update posteriors when a new task is introduced)
        """
        self.models_ = []
        self.X_ = []
        self.y_ = []
        self.n_tasks = 0
        self.n_classes = None
        
        if acorn is not None:
            np.random.seed(acorn)
    
    def new_forest(self, X, y, n_estimators=200, max_samples=0.32,
                        bootstrap=True, max_depth=30, min_samples_leaf=1,
                        acorn=None):
        """
        Input
        X: an array-like object of features; X.shape == (n_samples, n_features)
        y: an array-like object of class labels; len(y) == n_samples
        n_estimators: int; number of trees to construct (default = 200)
        max_samples: float in (0, 1]: number of samples to consider when 
            constructing a new tree (default = 0.32)
        bootstrap: bool; If True then the samples are sampled with replacement
        max_depth: int; maximum depth of a tree
        min_samples_leaf: int; minimum number of samples in a leaf node
        
        Return
        model: a BaggingClassifier fit to X, y
        """
        
        if X.ndim == 1:
            raise ValueError('1d data will cause headaches down the road')
            
        if acorn is not None:
            np.random.seed(acorn)
            
        self.X_.append(X)
        self.y_.append(y.astype(int))
            
        n = X.shape[0]
        K = len(np.unique(y))
        
        if self.n_classes is None:
            self.n_classes = K
        
        max_features = int(np.ceil(np.sqrt(X.shape[1])))

        model=BaggingClassifier(DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=min_samples_leaf,
                                                         max_features = max_features),
                                  n_estimators=n_estimators,
                                  max_samples=max_samples,
                                  bootstrap=bootstrap)

        model.fit(X, y)
        self.models_.append(model)
        self.n_tasks += 1
        self.n_classes = len(np.unique(y))
        
        return model
    
    
    def _get_leaves(self, estimator):
        """
        Internal function to get leaf node ids of estimator.
        
        Input
        estimator: a fit DecisionTreeClassifier
        
        Return
        leaf_ids: numpy array; an array of leaf node ids
        
        Usage
        _estimate_posteriors(..)
        """
        
        # adapted from https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
        n_nodes = estimator.tree_.node_count
        children_left = estimator.tree_.children_left
        children_right = estimator.tree_.children_right
        feature = estimator.tree_.feature
        threshold = estimator.tree_.threshold

        leaf_ids = []
        stack = [(0, -1)] 
        while len(stack) > 0:
            node_id, parent_depth = stack.pop()

            # If we have a test node
            if (children_left[node_id] != children_right[node_id]):
                stack.append((children_left[node_id], parent_depth + 1))
                stack.append((children_right[node_id], parent_depth + 1))
            else:
                leaf_ids.append(node_id)

        return np.array(leaf_ids)
    
    
    def _finite_sample_correction(self, class_probs, row_sums):
        """
        An internal function for finite sample correction of posterior estimation.
        
        Input
        class_probs: numpy array; array of posteriors to correct
        row_sums: numpy array; array of partition counts
        
        Output
        class_probs: numpy array; finite sample corrected posteriors
        
        Usage
        _estimate_posteriors(..)
        
        """
    
        where_0 = np.argwhere(class_probs == 0)
        for elem in where_0:
            class_probs[elem[0], elem[1]] = 1 / (2 * row_sums[elem[0], None])
        where_1 = np.argwhere(class_probs == 1)
        for elem in where_1:
            class_probs[elem[0], elem[1]] = 1 - 1 / (2 * row_sums[elem[0], None])
    
        return class_probs
    
    
    def _estimate_posteriors(self, test, representation=0, decider=0, subsample=1, acorn=None):
        """
        An internal function to estimate the posteriors.
        
        Input
        task_number: int; indicates which model in self.model_ to use
        test: array-like; test observation
        in_task: bool; True if test is an in-task observation(s)
        subsample: float in (0, 1]; proportion of out-of-task samples to use to
            estimate posteriors
            
        Return
        probs: numpy array; probs[i, k] is the probability of observation i
            being class k
            
        Usage
        predict(..)
        """
        
        if acorn is not None:
            acorn = np.random.seed(acorn)
            
        if representation==decider:
            in_task=True
        else:
            in_task=False
            
        train = self.X_[decider]
        y = self.y_[decider]
            
        model = self.models_[representation]

        n, d = train.shape
        
        if test.ndim > 1:
            m, d_ = test.shape
        else:
            m = len(test)
            d_ = 1

        class_counts = np.zeros((m, model.n_classes_))
        for idx, tree in enumerate(model):
            # get out of bag indicies
            
           
            if in_task:
                sampled_indices = model.estimators_samples_[idx]
                prob_indices = np.delete(range(n), sampled_indices)
            else:
                prob_indices = np.random.choice(range(n), size=int(subsample*n), replace=False)

            leaf_nodes = self._get_leaves(tree)
            unique_leaf_nodes = np.unique(leaf_nodes)

            # get all node counts
            node_counts = tree.tree_.n_node_samples
            # get probs for eval samples
            posterior_class_counts = np.zeros((len(unique_leaf_nodes), model.n_classes_))

            for prob_index in prob_indices:
                temp_node = tree.apply(train[prob_index].reshape(1, -1)).item()
                posterior_class_counts[np.where(unique_leaf_nodes == temp_node)[0][0], y[prob_index]] += 1

            # total number of points in a node
            row_sums = posterior_class_counts.sum(axis=1)

            # no divide by zero
            row_sums[row_sums == 0] = 1

            # posteriors
            class_probs = (posterior_class_counts / row_sums[:, None])
            # posteriors with finite sampling correction

            class_probs = self._finite_sample_correction(class_probs, row_sums)

            # posteriors as a list
            class_probs.tolist()

            partition_counts = np.asarray([node_counts[np.where(unique_leaf_nodes == x)[0][0]] for x in tree.apply(test)])
            # get probability for out of bag samples
            eval_class_probs = [class_probs[np.where(unique_leaf_nodes == x)[0][0]] for x in tree.apply(test)]
            eval_class_probs = np.array(eval_class_probs)
            # find total elements for out of bag samples
            elems = np.multiply(eval_class_probs, partition_counts[:, np.newaxis])
            # store counts for each x (repeat fhis for each tree)
            class_counts += elems
        # calculate p(y|X = x) for all x's
        probs = class_counts / class_counts.sum(axis=1, keepdims=True)

        return probs


    def predict(self, test, representation=0, decider='all', subsample=1, acorn=None):
        """
        Predicts the class labels for each sample in test.
        
        Input
        test: array-like; either a 1d array of length n_features
            or a 2d array of shape (m, n_features) 
        task_number: int; task number 
        """
        
        sum_posteriors = np.zeros((test.shape[0], self.n_classes))
        
        if representation is 'all':
            representation = np.arange(self.n_tasks)
        elif isinstance(representation, int):
            representation = np.array([representation])
        elif isinstance(representation, list):
            representation = np.array(representation)
            
        if not isinstance(representation, np.ndarray):
            raise ValueError('bad representation type %s: int, list of ints or numpy arrays only'%(str(type(representation)))
                            )
        else:
            representation = representation.astype(int)
            
        for i, rep in enumerate(representation):
            sum_posteriors += self._estimate_posteriors(test,
                                                        i,
                                                        decider,
                                                        subsample,
                                                        acorn)            
                
        return np.argmax(sum_posteriors, axis=1)

In [100]:
train_file = 'cifar-100-python/train'
unpickled_train = unpickle(train_file)
train_keys = list(unpickled_train.keys())
fine_labels = np.array(unpickled_train[train_keys[2]])

train_data = unpickled_train[list(train_keys)[-1]]
class_idx = [np.where(fine_labels == u)[0] for u in np.unique(fine_labels)]

train_by_task = [np.concatenate(class_idx[i*n_tasks: (i+1)*n_tasks]) for i in range(n_tasks)]

K = int(len(class_idx)/n_tasks)

n_trees = int(np.sqrt(len(class_idx[0])))

test_file = 'cifar-100-python/test'
unpickled_test = unpickle(test_file)
test_keys = list(unpickled_test.keys())
test_labels = np.array(unpickled_test[test_keys[2]])

test_data = unpickled_test[test_keys[-1]]
test_class_idx = [np.where(test_labels == u)[0] for u in np.unique(test_labels)]
test_by_task = [np.concatenate(test_class_idx[i*n_tasks: (i+1)*n_tasks]) for i in range(n_tasks)]

In [101]:
np.random.seed(1)

lifelong_forest = LifelongForest()

n=100
temp_n_tasks=2


for i in range(n_tasks):
    X = train_data[np.concatenate(class_idx[i*n_tasks: (i+1)*n_tasks])]
    labels = homogenize_labels(np.concatenate([n_tasks*i*np.ones(500) + j for j in range(n_tasks*i, n_tasks*(i+1))]))
    if i > 0:
        np.random.shuffle(labels)
    lifelong_forest.new_forest(X, labels)

In [102]:
stl_errors = np.zeros(n_tasks)
homogenized_labels = [homogenize_labels(test_labels[t]) for t in test_by_task[:n_tasks]]
llf_errors = [np.zeros((n_tasks-i)) for i in range(n_tasks)]

In [98]:
subsample=1

for i, test_set in enumerate(tqdm(test_by_task)):
    if i != 0:
        np.random.shuffle(homogenized_labels[i])
    for j in tqdm(range(i, n_tasks)):
        if i == j:
            stl_temp_pred = lifelong_forest.predict(test_data[test_set],
                                                representation=i,
                                                decider=i,
                                                subsample=subsample
                                                )
            stl_errors[i] = np.mean(stl_temp_pred == homogenized_labels[i])
            
            llf_temp_pred = lifelong_forest.predict(test_data[test_set],
                                                   representation=np.arange(i+1),
                                                   decider=i,
                                                   subsample=subsample
                                                   )
            llf_errors[i][i] = np.mean(stl_temp_pred == homogenized_labels[i])
        else:
            llf_temp_pred = lifelong_forest.predict(test_data[test_set],
                                                   representation=np.arange(j+1),
                                                   decider=i,
                                                   subsample=subsample
                                                   )
            llf_errors[i][j] = np.mean(llf_temp_pred == homogenized_labels[i])











  0%|          | 0/10 [00:00<?, ?it/s]










  0%|          | 0/10 [00:00<?, ?it/s]

10
0
0
1
0
2
0
4
0
6
0
7
0
8
0
9
0
10
0
11
0
13
0
14
0
15
0
16
0
17
0
18
0
19
0
20
0
21
0
22
0
23
0
24
0
25
0
26
0
27
0
28
0
29
0
31
0
33
0
34
0
37
0
39
0
40
0
41
0
42
0
45
0
46
0
48
0
49
0
50
0
51
0
52
0
54
0
55
0
56
0
57
0
58
0
59
0
61
0
63
0
64
0
66
0
67
0
68
0
69
0
70
0
72
0
74
0
75
0
76
0
77
0
78
0
79
0
80
0
82
0
83
0
84
0
85
0
87
0
89
0
90
0
91
0
92
0
93
0
95
0
96
0
97
0
98
0
102
0
104
0
105
0
107
0
108
0
109
0
110
0
112
0
114
0
115
0
117
0
118
0
119
0
120
0
121
0
122
0
123
0
124
0
126
0
127
0
128
0
129
0
130
0
131
0
132
0
133
0
134
0
136
0
137
0
138
0
140
0
142
0
143
0
144
0
147
0
148
0
149
0
152
0
153
0
154
0
155
0
156
0
157
0
160
0
161
0
162
0
163
0
164
0
165
0
166
0
167
0
168
0
169
0
171
0
173
0
174
0
178
0
179
0
180
0
181
0
183
0
184
0
188
0
189
0
190
0
192
0
193
0
194
0
195
0
196
0
197
0
199
0
200
0
201
0
202
0
203
0
204
0
206
0
207
0
208
0
209
0
211
0
212
0
213
0
217
0
218
0
219
0
220
0
221
0
224
0
226
0
227
0
228
0
229
0
230
0
231
0
233
0
234
0
235
0
236
0
237
0
238
0
239

1909
3
1913
3
1914
3
1915
3
1916
3
1917
3
1919
3
1920
3
1921
3
1923
3
1925
3
1927
3
1928
3
1930
3
1931
3
1932
3
1934
3
1935
3
1936
3
1937
3
1938
3
1943
3
1944
3
1945
3
1946
3
1947
3
1948
3
1949
3
1950
3
1952
3
1953
3
1954
3
1955
3
1957
3
1958
3
1959
3
1961
3
1963
3
1965
3
1966
3
1967
3
1968
3
1969
3
1970
3
1971
3
1972
3
1973
3
1975
3
1979
3
1980
3
1981
3
1982
3
1983
3
1984
3
1986
3
1987
3
1988
3
1989
3
1990
3
1991
3
1992
3
1993
3
1994
3
1995
3
1996
3
1997
3
1998
3
1999
3
2002
4
2003
4
2005
4
2006
4
2010
4
2011
4
2013
4
2014
4
2015
4
2016
4
2017
4
2018
4
2019
4
2020
4
2022
4
2024
4
2025
4
2027
4
2028
4
2029
4
2030
4
2031
4
2032
4
2034
4
2035
4
2036
4
2037
4
2039
4
2041
4
2042
4
2044
4
2047
4
2051
4
2052
4
2053
4
2054
4
2056
4
2057
4
2060
4
2061
4
2062
4
2063
4
2064
4
2066
4
2067
4
2068
4
2069
4
2070
4
2071
4
2073
4
2074
4
2075
4
2076
4
2077
4
2080
4
2082
4
2083
4
2084
4
2085
4
2086
4
2088
4
2089
4
2092
4
2093
4
2098
4
2099
4
2100
4
2104
4
2105
4
2106
4
2110
4
2111
4
2113
4
2115
4
2116
4

3582
7
3584
7
3585
7
3587
7
3588
7
3589
7
3591
7
3592
7
3594
7
3595
7
3597
7
3598
7
3599
7
3600
7
3601
7
3602
7
3603
7
3604
7
3605
7
3606
7
3607
7
3608
7
3609
7
3610
7
3611
7
3612
7
3613
7
3615
7
3616
7
3617
7
3618
7
3619
7
3620
7
3621
7
3623
7
3625
7
3626
7
3627
7
3628
7
3629
7
3630
7
3631
7
3633
7
3634
7
3638
7
3639
7
3640
7
3641
7
3642
7
3643
7
3644
7
3645
7
3646
7
3647
7
3648
7
3650
7
3651
7
3652
7
3654
7
3656
7
3660
7
3661
7
3662
7
3664
7
3665
7
3666
7
3667
7
3668
7
3670
7
3671
7
3672
7
3673
7
3674
7
3675
7
3676
7
3677
7
3678
7
3679
7
3680
7
3682
7
3688
7
3689
7
3690
7
3691
7
3692
7
3693
7
3694
7
3697
7
3698
7
3700
7
3701
7
3702
7
3703
7
3705
7
3706
7
3707
7
3709
7
3710
7
3711
7
3712
7
3713
7
3714
7
3715
7
3716
7
3718
7
3719
7
3721
7
3723
7
3724
7
3725
7
3726
7
3728
7
3730
7
3731
7
3732
7
3736
7
3737
7
3738
7
3739
7
3740
7
3743
7
3745
7
3746
7
3752
7
3753
7
3754
7
3755
7
3757
7
3759
7
3760
7
3761
7
3762
7
3763
7
3764
7
3766
7
3768
7
3769
7
3770
7
3771
7
3773
7
3774
7
3775
7
3776
7

0
302
0
303
0
304
0
305
0
306
0
307
0
308
0
311
0
312
0
313
0
314
0
315
0
316
0
317
0
318
0
319
0
320
0
322
0
323
0
325
0
326
0
327
0
328
0
329
0
330
0
331
0
332
0
333
0
336
0
337
0
338
0
339
0
341
0
342
0
343
0
344
0
346
0
349
0
350
0
351
0
352
0
354
0
355
0
358
0
359
0
360
0
362
0
363
0
364
0
365
0
366
0
369
0
371
0
374
0
375
0
377
0
378
0
379
0
380
0
381
0
382
0
384
0
385
0
387
0
388
0
390
0
391
0
392
0
393
0
396
0
400
0
401
0
402
0
403
0
404
0
405
0
406
0
408
0
409
0
411
0
412
0
413
0
416
0
417
0
418
0
419
0
420
0
421
0
422
0
423
0
424
0
425
0
426
0
427
0
430
0
434
0
435
0
438
0
439
0
440
0
441
0
442
0
443
0
446
0
447
0
449
0
450
0
451
0
452
0
455
0
456
0
457
0
458
0
459
0
460
0
461
0
463
0
464
0
465
0
467
0
468
0
469
0
471
0
472
0
473
0
474
0
475
0
476
0
477
0
479
0
482
0
485
0
486
0
487
0
488
0
489
0
490
0
491
0
492
0
493
0
494
0
495
0
496
0
497
0
498
0
499
0
500
1
501
1
502
1
503
1
505
1
506
1
508
1
509
1
510
1
515
1
519
1
520
1
521
1
522
1
524
1
525
1
529
1
530
1
531
1
532
1
53

KeyboardInterrupt: 